In [1]:
import os
from dotenv import load_dotenv
import pandas as pd
import pickle
import numpy as np
import os
import sys

sys.path.append(os.path.dirname(os.getcwd()))
from groq import Groq
from src.llm_engine.gemini_agent import Agent


In [2]:
import pandas as pd
df = pd.read_csv(r"./data/GridPuzzle_processed.csv")
# = df[~df["id"].isin([16, 244, 503, 687, 897, 919, 1020, 1084, 1250, 1370, 1597, 1792, 1935, 1980, 2078, 2140, 2208, 2300, 2500, 2550, 2566, 2675, 2691, 2700, 2723, 2847, 3002, 3444, 3458, 3510, 3601, 3624, 3642, 3736, 3936, 3954, 3957, 4020, 4064, 4289, 4465, 4600, 4624, 4658, 4937, 4941, 5152, 5212, 5257, 5325, 5585, 5628, 5659, 5740, 5772, 5804, 6045, 6125, 6292, 6402, 6421, 6432, 6541, 6621, 6671, 6746, 6748, 6782, 6823, 6843, 6925, 7098, 7116, 7183, 7188, 7234, 7389, 7404, 7429, 7591, 7598, 7610, 7685, 7729, 7865, 7866, 7943, 7959, 7976, 7993, 7997, 8066, 8142, 8198, 8356, 8458, 8474, 8481, 8526, 8600, 8685, 8712, 8716, 8853, 9039, 9043, 9065, 9076, 9121, 9139, 9288, 9390, 9476, 9497, 9560, 9596, 9719, 9738, 9779, 9965, 9976, 9984, 10037, 10100, 10122, 10123, 10170, 10178, 10251, 10440, 10450, 10564, 10583, 10730, 11022, 11033, 11108, 11163, 11214, 11263, 11304, 11310, 11321, 11367, 11407, 11434, 11465, 11474, 11546, 11592, 11603, 11732, 11748, 11908, 11923, 11954, 12037, 12082, 12117, 12133, 12172, 12221, 12356, 12395, 12535, 12586, 12595, 12652, 12689, 12702, 12703, 12782, 12818, 12900, 12980, 13003, 13048, 13051, 13198, 13364, 13407, 13422, 13458,13527,13561])]
# df = df[df["id"].isin([13, 821, 687, 718, 350, 1084, 1677, 1437, 1597, 1935, 1020, 1495, 2893, 2208, 2700, 2788, 3720, 3936, 3642, 3659, 4465, 4658, 4087, 4861, 4140, 4464, 4848, 3954, 5740, 5675, 5212, 4976, 5889, 5659, 5325, 5585, 6843, 6292, 6045, 6432, 6823, 5923, 6778, 6997, 7685, 7865, 7183, 7389, 7866, 7188, 7837, 8712, 7959, 8356, 8142, 7993, 8519, 9106, 9719, 9139, 9487, 8853, 8988, 9043, 9497, 9596, 9965, 10037, 10380, 10100, 10450, 10440, 9976, 10251, 11592, 11097, 11022, 11603, 11546, 11748, 11263, 11367, 11407, 11304, 11321, 11923, 12703, 12356, 12535, 12172, 12133, 12395, 12818, 13003, 12900, 13407, 13198, 12782, 13051, 12980])]
df.head(1)

,key,id,question,answer,Mistral-7b,Llama-13b,gemini-pro,gpt-4-turbo,Claude-3,actual_question,Clues,categories,number_of_clues,number_categories,category_names
0,341.json,802,Nicholas and Edith are having a small dinner p...,1984 | Annata Branco | gewurztraminer\n1988 | ...,| 1984 | Annata Branco | gewurztraminer |\n |-...,Let's solve the puzzle step by step.\n \n Step...,"Step-by-step solution:\n 1. From clue 6, we kn...","Step-by-step solution:\n \n 1. From clue 6, we...","Step-by-step solution:\n \n 1. From clue 6, we...",Nicholas and Edith are having a small dinner p...,Clues:\n1. The Ece Suss was bottled sometime a...,"vintages : 1984, 1988, 1992, 1996.\nwines : An...",6,3,vintages wines types


In [5]:
import pandas as pd
df = pd.read_csv(r"../GridPuzzle_final.csv")
# = df[~df["id"].isin([16, 244, 503, 687, 897, 919, 1020, 1084, 1250, 1370, 1597, 1792, 1935, 1980, 2078, 2140, 2208, 2300, 2500, 2550, 2566, 2675, 2691, 2700, 2723, 2847, 3002, 3444, 3458, 3510, 3601, 3624, 3642, 3736, 3936, 3954, 3957, 4020, 4064, 4289, 4465, 4600, 4624, 4658, 4937, 4941, 5152, 5212, 5257, 5325, 5585, 5628, 5659, 5740, 5772, 5804, 6045, 6125, 6292, 6402, 6421, 6432, 6541, 6621, 6671, 6746, 6748, 6782, 6823, 6843, 6925, 7098, 7116, 7183, 7188, 7234, 7389, 7404, 7429, 7591, 7598, 7610, 7685, 7729, 7865, 7866, 7943, 7959, 7976, 7993, 7997, 8066, 8142, 8198, 8356, 8458, 8474, 8481, 8526, 8600, 8685, 8712, 8716, 8853, 9039, 9043, 9065, 9076, 9121, 9139, 9288, 9390, 9476, 9497, 9560, 9596, 9719, 9738, 9779, 9965, 9976, 9984, 10037, 10100, 10122, 10123, 10170, 10178, 10251, 10440, 10450, 10564, 10583, 10730, 11022, 11033, 11108, 11163, 11214, 11263, 11304, 11310, 11321, 11367, 11407, 11434, 11465, 11474, 11546, 11592, 11603, 11732, 11748, 11908, 11923, 11954, 12037, 12082, 12117, 12133, 12172, 12221, 12356, 12395, 12535, 12586, 12595, 12652, 12689, 12702, 12703, 12782, 12818, 12900, 12980, 13003, 13048, 13051, 13198, 13364, 13407, 13422, 13458,13527,13561])]
# df = df[df["id"].isin([13, 821, 687, 718, 350, 1084, 1677, 1437, 1597, 1935, 1020, 1495, 2893, 2208, 2700, 2788, 3720, 3936, 3642, 3659, 4465, 4658, 4087, 4861, 4140, 4464, 4848, 3954, 5740, 5675, 5212, 4976, 5889, 5659, 5325, 5585, 6843, 6292, 6045, 6432, 6823, 5923, 6778, 6997, 7685, 7865, 7183, 7389, 7866, 7188, 7837, 8712, 7959, 8356, 8142, 7993, 8519, 9106, 9719, 9139, 9487, 8853, 8988, 9043, 9497, 9596, 9965, 10037, 10380, 10100, 10450, 10440, 9976, 10251, 11592, 11097, 11022, 11603, 11546, 11748, 11263, 11367, 11407, 11304, 11321, 11923, 12703, 12356, 12535, 12172, 12133, 12395, 12818, 13003, 12900, 13407, 13198, 12782, 13051, 12980])]
df.head(1)

,,,,,,,,,,,,,,,GridPuzzle_processed
key,id,question,answer,Manual Validation as of 25th April (Pushkar),Mistral-7b,Llama-13b,gemini-pro,gpt-4-turbo,Claude-3,actual_question,Clues,categories,number_of_clues,number_categories,category_names


In [3]:
df[df["id"] == 997]

,key,id,question,answer,Mistral-7b,Llama-13b,gemini-pro,gpt-4-turbo,Claude-3,actual_question,Clues,categories,number_of_clues,number_categories,category_names


In [47]:
df.shape

(89, 15)

In [48]:
v = [11304, 13407, 3444]
ids_ = df["id"].values.tolist()

for val in ids_:
    if val in v:
        print(val)

In [19]:
print(df[df["id"] == 244]["answer"].values.tolist()[0])

April 4th | Linda Lott | solar eclipse
April 5th | Randy Reid | flooding
April 6th | Tam Terry | earthquake
April 7th | Sid Snyder | royal wedding


In [ ]:
# Final Answer:


# April 4th | Linda Lott | earthquake
# April 5th | Tam Terry | flooding
# April 6th | Sid Snyder | solar eclipse
# April 7th | Randy Reid | royal wedding


In [37]:
import shutil

path = r"C:\Users\aniru\Downloads\NLP- Project\deepseek_responses"

dest = r"C:\Users\aniru\Downloads\validate"

indices = [16, 244, 503, 687, 897, 919, 1020, 1084, 1250, 1370, 1597, 1792, 1935, 1980, 2078, 2140, 2208, 2300, 2500, 2550, 2566, 2675, 2691, 2700, 2723, 2847, 3002, 3444, 3458, 3510, 3601, 3624, 3642, 3736, 3936, 3954, 3957, 4020, 4064, 4289, 4465, 4600, 4624, 4658, 4937, 4941, 5152, 5212, 5257, 5325, 5585, 5628, 5659, 5740, 5772, 5804, 6045, 6125, 6292, 6402, 6421, 6432, 6541, 6621, 6671, 6746, 6748, 6782, 6823, 6843, 6925, 7098, 7116, 7183, 7188, 7234, 7389, 7404, 7429, 7591, 7598, 7610, 7685, 7729, 7865, 7866, 7943, 7959, 7976, 7993, 7997, 8066, 8142, 8198, 8356, 8458, 8474, 8481, 8526, 8600, 8685, 8712, 8716, 8853, 9039, 9043, 9065, 9076, 9121, 9139, 9288, 9390, 9476, 9497, 9560, 9596, 9719, 9738, 9779, 9965, 9976, 9984, 10037, 10100, 10122, 10123, 10170, 10178, 10251, 10440, 10450, 10564, 10583, 10730, 11022, 11033, 11108, 11163, 11214, 11263, 11304, 11310, 11321, 11367, 11407, 11434, 11465, 11474, 11546, 11592, 11603, 11732, 11748, 11908, 11923, 11954, 12037, 12082, 12117, 12133, 12172, 12221, 12356, 12395, 12535, 12586, 12595, 12652, 12689, 12702, 12703, 12782, 12818, 12900, 12980, 13003, 13048, 13051, 13198, 13364, 13407, 13422, 13458,13527,13561]

for file in os.listdir(path):

    index = int(file.split("_")[2].split(".")[0])

    if index in indices:

        shutil.copy(os.path.join(path, file), dest)
        


In [40]:
import re
path = r"C:\Users\aniru\Downloads\validate"
valid = []
for file in os.listdir(path):

    if "table" not in file:

        with open(os.path.join(path, file), "r") as f:
            lines = f.readlines()
        
        reverse_lines = lines[::-1]

        ckpt = -1
        
        for line_index, line in enumerate(reverse_lines):
            
            if ckpt == 1:
                break

            elif ckpt == -1 and (line_index / len(reverse_lines)) >= 0.25:
                break

            else:
                if re.search(r"final solution|final table|final answer", line.lower()) is not None:
                    ckpt = 1

        if ckpt == 1:
            valid.append(os.path.join(path, file))


print(len(valid))

83


In [35]:
valid[1]

'C:\\Users\\aniru\\Downloads\\validate\\deepseek_pred_4658.txt'

In [41]:
path = r"C:\Users\aniru\Downloads\validate"

dest_1 = r"C:\Users\aniru\Downloads\validate\with_tables"
dest_2 = r"C:\Users\aniru\Downloads\validate\without_tables"



if not os.path.exists(dest_1):
    os.mkdir(dest_1)

if not os.path.exists(dest_2):
    os.mkdir(dest_2)


for file in os.listdir(path):

    if not os.path.isdir(os.path.join(path,file)):
    
        if os.path.join(path, file) in valid:
            
            shutil.move(os.path.join(path, file), dest_1)
        
        else:
            shutil.move(os.path.join(path, file), dest_2)


In [3]:
client = Groq(
    api_key= os.getenv("GROQ_API_KEY"),
)

In [3]:
import re
def process_ls(ls):
    if re.search(r"\d+",ls[0]) is None:
        return ls

    else:
        res = []
        for val in ls:
            val = re.sub(r"[^\d+]", '', val)
            res.append(val)
        return res


In [4]:
import re
def process_question(question):
    lines = question.split("\n")
    categories = {}
    clues = []
    actual_question = ""
    ckpt = -1
    flag = -1
    for index, line in enumerate(lines):


        if ckpt == 1 and re.search(r"^\d+\.", line) is None:
            break

        if " : " in line:
            if flag == -1:
                flag = index
            title, ls = line.split(" : ")[0], line.split(" : ")[1].split(", ")
            ls = process_ls(ls)
            categories[title] = ls
        
        if "Clues" in line:
            ckpt = 1
            continue
        
        if ckpt == 1 and re.search(r"\d+\.", line) is not None:
            clues.append(line)
        


    actual_question += "\n".join(lines[:flag])            

    return actual_question, categories, clues


#ques, categories, clues = process_question(df["question"].iloc[100])


In [6]:
path = r"./deepseek_responses"
if not os.path.exists(path):
    os.mkdir(path)

In [10]:
for i, row in df.iterrows():
    
    id_ = row["id"]
    user_query = row["question"]


    completion = client.chat.completions.create(
        model="deepseek-r1-distill-llama-70b",
        messages=[{
                "role": "user",
                "content": user_query

            }],
        temperature=0,
        max_completion_tokens=8192,
        top_p=1,
        stream=True,
        stop=None,
    )

    groq_deepseek_response = ""
    for chunk in completion:
        text = chunk.choices[0].delta.content or ""
        groq_deepseek_response += text

    groq_deepseek_response


    with open(os.path.join(path, f"deepseek_pred_{id_}.txt"), "w") as f:
        f.write(groq_deepseek_response)

In [11]:
path = r"./deepseek_responses"
if not os.path.exists(path):
    os.mkdir(path)

In [5]:
ids= []
for file in os.listdir(path):

    id_ = file.split("_")[2].split(".")[0]

    ids.append(id_)


In [6]:
print(len(ids))

132


In [9]:
df["id" ] = df["id"].astype(str)

df = df[~df["id"].isin(ids)]

print(df.shape)

(142, 15)


In [16]:
agent = Agent(system_prompt="You are an AI assistant", agent_name="csp_agent")

for i, row in df.iterrows():
    
    id_ = row["id"]
    
    user_query = row["question"]

    response = agent.perform_action(user_query=user_query)


    with open(os.path.join(path, f"gemini_pred_{id_}.txt"), "w") as f:
        f.write(response)

In [4]:
import os
import re

count = 0
path = r"./gemini_responses"
for file in os.listdir(path):
    fp = os.path.join(path, file)

    with open(fp, "r") as f:
        lines = f.readlines()
    
    text = "\n".join(lines)
    if re.search(r"final answer:", text.lower()) is not None:

        reqd_ = text.split("Final Answer:")[1]
        
        if reqd_:
            count += 1

print(count)

14
